In [ ]:
# Set the current working directory to the root of the repo (`CWD` is set in the Makefile)
import os
os.chdir(os.environ['CWD'])
%pwd

In [ ]:
import datetime as dt

import sqlalchemy as sa
from sqlalchemy import select
from sqlalchemy.orm import sessionmaker
import pandas as pd

import altair as alt
alt.data_transformers.enable('default', max_rows=None)

import pvsite_datamodel.sqlmodels as models

In [ ]:
# This is the host given my ssh tunnel. The password is set in a .pgpass file.
HOST='postgresql://main@localhost:9997/pvsitedevelopment'

START_UTC = dt.datetime.utcnow() - dt.timedelta(days=1)
END_UTC = dt.datetime.utcnow() + dt.timedelta(hours=36)
HORIZON_MINUTES = 2 * 60# 60 * 1

In [ ]:
# `future=True` to try out sqlalchemy's 2.0 syntax (that also work in 1.4)
engine = sa.create_engine(HOST, future=True)
Session = sessionmaker(engine)

In [ ]:
# How many sites in the DB
with Session() as session:
    num_sites = session.scalars(
        select(sa.func.count()).select_from(models.SiteSQL)
    ).one()
print(num_sites)

In [ ]:
# Find sites with some predictions
with Session() as session:
    site_uuids = session.scalars(
        select(models.ForecastSQL.site_uuid).distinct().order_by()
    ).all()
print(len(site_uuids))

In [ ]:
offset = 0
limit = 100
site_uuids = site_uuids[offset: offset + limit]
#site_uuids

In [ ]:
def rows_to_df(query, columns=None):
    if columns is None:
        query = query.all()
        columns = list(query[0].keys())
    data = [
        {
            key: getattr(row, key)
            for key
            in columns
        }
        for row
        in query
    ]
    
    df = pd.DataFrame.from_records(data)
    
    # Change the types on some columns based on the name.
    for col in columns:
        if col.endswith('_utc'):
            df[col] = pd.to_datetime(df[col])
        elif col.endswith('_uuid'):
            df[col] = df[col].astype(str)
        elif col.endswith('_power_kw'):
            df['power_kw'] = df[col]
            del df[col]
    
    return df

In [ ]:
# Get the recent generation for the selected sites
with Session() as session:
    query = session.execute(
        select(
            models.SiteSQL.client_site_id,
            models.GenerationSQL.generation_power_kw,
            models.GenerationSQL.start_utc,
            models.GenerationSQL.end_utc,
        )
        .join(models.GenerationSQL.site)
        .where(models.GenerationSQL.site_uuid.in_(site_uuids))
        .where(models.GenerationSQL.start_utc >= START_UTC)
        .where(models.GenerationSQL.start_utc < END_UTC)
        #.where(models.GenerationSQL.generation_power_kw > 0)
    )
    
    df_generation = rows_to_df(query)
df_generation.head()

In [ ]:
# Get the corresponding forecasts
# This query still takes a few seconds to run.
# TODO: understand why and make it faster.
with Session() as session:
    query = session.execute(
        select(
            models.ForecastValueSQL.forecast_power_kw,
            models.ForecastValueSQL.start_utc,
            models.ForecastValueSQL.end_utc,
            models.SiteSQL.client_site_id,
        )
        .select_from(models.ForecastValueSQL)
        .join(models.ForecastSQL)
        .join(models.SiteSQL)
        .where(models.ForecastSQL.site_uuid.in_(site_uuids))
        #.where(models.ForecastValueSQL.forecast_power_kw > 0)
        .where(models.ForecastValueSQL.horizon_minutes == HORIZON_MINUTES)
        .where(models.ForecastValueSQL.start_utc >= START_UTC)
        .where(models.ForecastValueSQL.start_utc < END_UTC)
     #   .limit(10)
    )
    df_forecast = rows_to_df(query)
df_forecast.head()

In [ ]:
df_generation['which'] = 'generation'
df_forecast['which'] = 'forecast'
df = pd.concat([df_forecast, df_generation])

In [ ]:
df.head()

In [ ]:
data = df.copy()#[:1000]
ids = data['client_site_id'].unique()[:100]
data = data[data['client_site_id'].isin(ids)]
data = data.sort_values('which', ascending=False)
data['timestamp'] = data['start_utc'] + (data['end_utc'] - data['start_utc']) / 2
#data = data[data['power_kw'] > 0]
base_generation = (
    alt.Chart()#data[data['which'] == 'generation'])
    .mark_line(color='black')#size=10, opacity=0.5, color='black')
    .encode(x='timestamp', y='power_kw')
    .transform_filter(alt.datum.which == 'generation')
    .properties(height=100, width=200)
        

#     .encode(
#         x='timestamp',
#         y='power_kw',
        #facet=alt.Facet('client_site_id', columns=6),
        #color=alt.Color('which', scale=alt.Scale(domain=['forecast', 'generation'], range=['red', 'black']))
    )


#)

base_forecast = (
    alt.Chart()#data[data['which'] == 'forecast'])
    .mark_line(color='orange')#size=25, opacity=0.5, color='orange')
    .encode(x='timestamp', y='power_kw')
    .transform_filter(alt.datum.which == 'forecast')
)

chart = alt.layer(base_generation, base_forecast, data=data).facet('client_site_id', columns=10).resolve_scale(
    y="independent",
)

    
chart